In [2]:
%load_ext autoreload
%autoreload 1
%matplotlib inline

import os
import shutil
import sys
import subprocess

%aimport cnv_inference_config
project_config = cnv_inference_config
os.chdir(project_config.MB_ROOT)

from collections import defaultdict, OrderedDict
import sys

import numpy as np
import pandas as pd
from tqdm import tqdm, tqdm_notebook

%aimport util
%aimport workspace_manager

In [3]:
workspace = workspace_manager.WorkspaceManager(
    task_name="counts_to_ase",
    experiment_info={
        "data" : "scDNA",
        "sample" : "first_sample"
    },
    cookiecutter_info=util.load_cookiecutter_info(project_config),
    verbose=True
)
requirements = {
        dtype : f"{dtype}.pkl"
        for dtype in ["raw_counts", "phasing", 
                      "genome", "counts", 
                      "gene_counts", "scoring", 
                      "snp_to_genes", "gene_to_snps"]
    }
requirements.update({
    "clustering" : "cnv_3_clustering.pkl"
})
workspace.prepare_workspace(requirements)

processing read-only data directory
processing workspace directory
processing image directory
loading data to workspace directory


/icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference/data/processed/first_sample/scDNA/raw_counts.pkl —> /icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference/data/tmp/counts_to_ase/first_sample/scDNA/raw_counts.pkl
/icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference/data/processed/first_sample/scDNA/phasing.pkl —> /icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference/data/tmp/counts_to_ase/first_sample/scDNA/phasing.pkl
/icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference/data/processed/first_sample/scDNA/genome.pkl —> /icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference/data/tmp/counts_to_ase/first_sample/scDNA/genome.pkl
/icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference/data/processed/first_sample/scDNA/counts.pkl —> /icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference/data/tmp/counts_to_ase/first_sample/scDNA/counts.pkl
/icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference/data/processed/first_sample/scDNA/gene_counts.pkl —> /icgc/dkfzlsdf/analysis/B260/users/v39

/icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference/data/processed/first_sample/scDNA/cnv_3_clustering.pkl —> /icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference/data/tmp/counts_to_ase/first_sample/scDNA/clustering.pkl


In [4]:
workspace

Task name: counts_to_ase
Experiment info:
{
    "data": "scDNA",
    "sample": "first_sample"
}
Config generated on 2019-08-14 15:05
Contents:
{
    "task_name": "counts_to_ase",
    "experiment_info": {
        "data": "scDNA",
        "sample": "first_sample"
    },
    "cookiecutter_info": {
        "root": "/icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference",
        "lib": "/icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference/lib",
        "notebooks": "/icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference/notebooks",
        "src": "/icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference/src",
        "img": "/icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference/img",
        "data": "/icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference/data",
        "raw": "/icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference/data/raw",
        "tmp": "/icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference/data/tmp",
        "processed": "/icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference/data/processed"
    },
    "config_dir": "/icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference/notebooks/counts_to_ase",
    "data_dir": {
        "root": "/icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference/data/processed",
        "sample": "/icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference/data/processed/first_sample/scDNA"
    },
    "data": {
        "raw_counts": "/icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference/data/processed/first_sample/scDNA/raw_counts.pkl",
        "phasing": "/icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference/data/processed/first_sample/scDNA/phasing.pkl",
        "genome": "/icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference/data/processed/first_sample/scDNA/genome.pkl",
        "counts": "/icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference/data/processed/first_sample/scDNA/counts.pkl",
        "gene_counts": "/icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference/data/processed/first_sample/scDNA/gene_counts.pkl",
        "scoring": "/icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference/data/processed/first_sample/scDNA/scoring.pkl",
        "snp_to_genes": "/icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference/data/processed/first_sample/scDNA/snp_to_genes.pkl",
        "gene_to_snps": "/icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference/data/processed/first_sample/scDNA/gene_to_snps.pkl",
        "clustering": "/icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference/data/processed/first_sample/scDNA/cnv_3_clustering.pkl"
    },
    "tmp_data_dir": {
        "root": "/icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference/data/tmp/counts_to_ase",
        "sample": "/icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference/data/tmp/counts_to_ase/first_sample/scDNA"
    },
    "tmp_data": {
        "raw_counts": "/icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference/data/tmp/counts_to_ase/first_sample/scDNA/raw_counts.pkl",
        "phasing": "/icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference/data/tmp/counts_to_ase/first_sample/scDNA/phasing.pkl",
        "genome": "/icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference/data/tmp/counts_to_ase/first_sample/scDNA/genome.pkl",
        "counts": "/icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference/data/tmp/counts_to_ase/first_sample/scDNA/counts.pkl",
        "gene_counts": "/icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference/data/tmp/counts_to_ase/first_sample/scDNA/gene_counts.pkl",
        "scoring": "/icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference/data/tmp/counts_to_ase/first_sample/scDNA/scoring.pkl",
        "snp_to_genes": "/icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference/data/tmp/counts_to_ase/first_sample/scDNA/snp_to_genes.pkl",
        "gene_to_snps": "/icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference/data/tmp/counts_to_ase/first_sample/scDNA/gene_to_snps.pkl",
        "clustering": "/icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference/data/tmp/counts_to_ase/first_sample/scDNA/clustering.pkl"
